In [10]:
from datetime import datetime, timezone
import pandas as pd
from dyson_pool import DysonPool
from time_utils import compute_due_time_and_duration, tm


def print_pool_state(pool, label):
    snapshot = pool.snapshot(day=tm.getTimestamp() / 86400, price=2000)
    print(f"\n\U0001F4D8 {label} Pool State")
    df = pd.DataFrame([{
        "ETH Reserve": round(snapshot['reserve_eth'], 6),
        "USDC Reserve": round(snapshot['reserve_usdc'], 6),
        "K": round(snapshot['k'], 6),
        "w": pool.w,
    }])
    print(df.to_markdown(index=False))


def print_note_summary(title, in0, in1, note0, note1, note0_with_premium, note1_with_premium, prem_ratio, due, duration_sec, q_old, q_new, option_type=None):
    print(f"\n\U0001F4C4 {title}")
    # Create base data as list of tuples
    data_items = [
        ("Input ETH", in0),
        ("Input USDC", in1),
    ]

    # Insert pool swap info at position 2
    if option_type == "put":
        data_items.extend(
            [
                ("Pool Swap In\n(User Real Input)", f"{note0_with_premium:.6f} ETH"),
                ("Pool Swap Out", f"{note1} USDC"),
            ]
        )
    elif option_type == "call":
        data_items.extend(
            [
                ("Pool Swap In\n(User Real Input)", f"{note1_with_premium:.6f} USDC"),
                ("Pool Swap Out", f"{note0} ETH"),
            ]
        )

    # Add remaining data
    data_items.extend(
        [
            ("Note ETH", round(note0, 6)),
            ("Note USDC", round(note1, 6)),
            ("Note(+Premium) ETH", round(note0_with_premium, 6)),
            ("Note(+Premium) USDC", round(note1_with_premium, 6)),
            ("Premium Ratio", round(prem_ratio, 6)),
            ("Due Day", due),
            ("Duration (s)", duration_sec),
            ("Q Old", round(q_old, 6)),
            ("Q New", round(q_new, 6)),
        ]
    )

    # Convert to DataFrame
    df = pd.DataFrame([dict(data_items)])
    print(df.to_markdown(index=False))


def print_withdraw_result(notes):
    print("\n\U0001F4B8 Withdraw Results")
    rows = [{
        "Note ID": n.id,
        "Returned ETH": round(amt0, 6),
        "Returned USDC": round(amt1, 6)
    } for n, amt0, amt1 in notes]
    print(pd.DataFrame(rows).to_markdown(index=False))


def test_forward_deposit():
    print("\n=== Testing Dual Investment Deposit ===")
    pool = DysonPool(init_eth=100.0, init_usdc=200000.0, basis=0.5, w_factor=2)
    current_time = datetime(2025, 5, 16, 23, 37, tzinfo=timezone.utc)
    tm.setCurrentTime(current_time)
    today = 1
    price = 2000

    print_pool_state(pool, "Initial")

    in0, in1 = 0, 2000
    lock_days = 30
    nid, note0, note1, note0_with_premium, note1_with_premium, prem_ratio, due, duration_sec, q_old, q_new = pool.deposit(in0, in1, lock_days, today, price)

    print_note_summary("Forward Deposit Result", in0, in1, note0, note1, note0_with_premium, note1_with_premium, prem_ratio, due, duration_sec, q_old, q_new)
    print_pool_state(pool, "After Forward Deposit")

    withdraw_time = datetime(2025, 6, 17, 23, 37, tzinfo=timezone.utc)
    tm.setCurrentTime(withdraw_time)
    withdrawn = pool.withdraw_due(withdraw_time, price)

    print_withdraw_result(withdrawn)
    print_pool_state(pool, "After Withdrawal")
    tm.resetMock()


def test_reverse_deposit_and_exercise():
    pool = DysonPool(init_eth=100.0, init_usdc=200000.0, basis=0.5, w_factor=1)
    current_time = datetime(2025, 5, 16, 23, 37, tzinfo=timezone.utc)
    exercise_time = datetime(2025, 5, 20, 12, 0, tzinfo=timezone.utc)
    today = 1
    price = 2000

    print_pool_state(pool, "Initial")

    lock_days = 30

    ### Reverse Deposit - Put
    option_type = "put"
    in0, in1 = 1, 0
    tm.setCurrentTime(current_time)
    nid_put, note0, note1, note0_with_premium, note1_with_premium, prem_ratio, due, duration_sec, q_old, q_new = pool.reverse_deposit(in0, in1, lock_days, today, option_type, price)

    print_note_summary("Reverse Deposit (Put)", in0, in1, note0, note1, note0_with_premium, note1_with_premium, prem_ratio, due, duration_sec, q_old, q_new, option_type=option_type)
    print_pool_state(pool, "After Reverse Deposit (Put)")

    tm.setCurrentTime(exercise_time)
    pool.exercise_option(nid_put, exercise_time, price)
    print(f"\n\U0001F4AA Exercised Put: User paid {note0_with_premium:.6f} ETH to receive {note1_with_premium:.6f} USDC")
    print_pool_state(pool, "After Exercising Put Option")

    ### Reverse Deposit - Call
    option_type = "call"
    in0, in1 = 0, 2000
    tm.setCurrentTime(current_time)
    nid_call, note0, note1, note0_with_premium, note1_with_premium, prem_ratio, due, duration_sec, q_old, q_new = pool.reverse_deposit(in0, in1, lock_days, today, option_type, price)

    print_note_summary("Reverse Deposit (Call)", in0, in1, note0, note1, note0_with_premium, note1_with_premium, prem_ratio, due, duration_sec, q_old, q_new, option_type=option_type)
    print_pool_state(pool, "After Reverse Deposit (Call)")

    tm.setCurrentTime(exercise_time)
    pool.exercise_option(nid_call, exercise_time, price)
    print(f"\n\U0001F4AA Exercised Call: User paid {note1_with_premium:.6f} USDC to receive {note0_with_premium:.6f} ETH")
    print_pool_state(pool, "After Exercising Call Option")

    tm.resetMock()

# 正向雙幣

In [11]:
test_forward_deposit()


=== Testing Dual Investment Deposit ===

📘 Initial Pool State
|   ETH Reserve |   USDC Reserve |       K |       w |
|--------------:|---------------:|--------:|--------:|
|           100 |         200000 | 4472.14 | 8944.27 |
-----------note0 0.9950310328778279
-----------note1 2000.0

📄 Forward Deposit Result
|   Input ETH |   Input USDC |   Note ETH |   Note USDC |   Note(+Premium) ETH |   Note(+Premium) USDC |   Premium Ratio |   Due Day |   Duration (s) |   Q Old |   Q New |
|------------:|-------------:|-----------:|------------:|---------------------:|----------------------:|----------------:|----------:|---------------:|--------:|--------:|
|           0 |         2000 |   0.995031 |        2000 |              1.05196 |               2114.42 |        0.057211 |     20255 |    2.59338e+06 |       0 | 44.6101 |

📘 After Forward Deposit Pool State
|   ETH Reserve |   USDC Reserve |       K |       w |
|--------------:|---------------:|--------:|--------:|
|         100.5 |       

# 反向雙幣

In [12]:
test_reverse_deposit_and_exercise()


📘 Initial Pool State
|   ETH Reserve |   USDC Reserve |       K |       w |
|--------------:|---------------:|--------:|--------:|
|           100 |         200000 | 4472.14 | 4472.14 |
-----------note0 1
-----------note1 1980.1980198019803

📄 Reverse Deposit (Put)
|   Input ETH |   Input USDC | Pool Swap In        | Pool Swap Out           |   Note ETH |   Note USDC |   Note(+Premium) ETH |   Note(+Premium) USDC |   Premium Ratio |   Due Day |   Duration (s) |   Q Old |    Q New |
|             |              | (User Real Input)   |                         |            |             |                      |                       |                 |           |                |         |          |
|------------:|-------------:|:--------------------|:------------------------|-----------:|------------:|---------------------:|----------------------:|----------------:|----------:|---------------:|--------:|---------:|
|           1 |            0 | 1.115867 ETH        | 1980.198019801980